In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from influxdb import *
import datetime


In [2]:
INFLUX_DB_IP = '10.12.97.178'
INFLUX_DB_PORT = 8086
INFLUX_DB = 'demoDb'
TARGET_MEASUREMENT = 'EM_main_new'

con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
query = 'select  "time", "DeviceID","EM_TOTAL_Import_Energy(kWh)"  from ' + TARGET_MEASUREMENT + ' where time > now() - 4d '
df1 = pd.DataFrame(con_obj.query(query, chunked=True, chunk_size=10000).get_points())
df1['time'] = df1['time'].astype('datetime64[ns]')


In [3]:
df1.head()

,time,DeviceID,EM_TOTAL_Import_Energy(kWh)
0,2020-06-22 03:38:49.385500928,EM10,140891.5
1,2020-06-22 03:38:49.393491968,EM5,282689.0
2,2020-06-22 03:38:49.989245952,EM9,36053.2
3,2020-06-22 03:38:50.148081920,EM1,585806.0
4,2020-06-22 03:38:50.641288960,EM7,39.8


In [4]:
df1['time'].min()

Timestamp('2020-06-22 03:38:49.385500928')

In [5]:
df1['time'].max()

Timestamp('2020-06-26 02:59:11.462104064')

In [6]:
date_old = pd.datetime.now().date() - pd.offsets.Day(3)
date_now = pd.datetime.now().date() - pd.offsets.Day(0)
df1 = df1[(df1['time'] >= date_old) & (df1['time'] < date_now)]



In [7]:
df1['time'].min()

Timestamp('2020-06-23 00:00:00.118753024')

In [8]:
df1['time'].max()

Timestamp('2020-06-25 23:59:59.887915008')

In [30]:
# INFLUX_DB_IP = '10.12.97.178'
# INFLUX_DB_PORT = 8086
# INFLUX_DB = 'demoDb'
# TARGET_MEASUREMENT = 'EM_live'

# con_obj = InfluxDBClient(host=INFLUX_DB_IP, port=INFLUX_DB_PORT, database=INFLUX_DB)
# query = 'select  "time", "DeviceID","EM_TOTAL_Import_Energy(kWh)"  from ' + TARGET_MEASUREMENT + ' where time > now() - 2d '
# df2 = pd.DataFrame(con_obj.query(query, chunked=True, chunk_size=10000).get_points())
# df2['time'] = df2['time'].astype('datetime64[ns]')


In [31]:
# date_old = pd.datetime.now().date() - pd.offsets.Day(1)
# date_now = pd.datetime.now().date() - pd.offsets.Day(0)
# df2 = df2[(df2['time'] >= date_old) & (df2['time'] < date_now)]



In [32]:
# df3 = pd.concat([df1, df2], ignore_index = True).drop_duplicates('time')


In [33]:
# df3.head(2)

,time,DeviceID,EM_TOTAL_Import_Energy(kWh)
0,2020-06-23 00:00:00.118753024,EM9,36698.3
1,2020-06-23 00:00:00.197954048,EM5,287249.0


In [35]:
# df3.time.max()

Timestamp('2020-06-25 23:59:59.887915008')

## Grid

In [25]:
grid=df1.copy()
grid=grid.set_index('time')
grid = grid[grid['DeviceID'] == 'EM1']
grid['diff']=grid.index.to_series().diff().dt.total_seconds().dropna()
#extra['differ']=extra.groupby(extra.index.to_series().diff().total_seconds())

In [26]:
grid['diff'].describe()

count    55465.000000
mean         4.673157
std        162.902588
min          0.625076
25%          2.264503
50%          2.449948
75%          2.659312
max      26861.158005
Name: diff, dtype: float64

In [29]:
grid[grid['diff'] > 900]['diff']


time
2020-06-23 08:15:57.657063936     2215.175607
2020-06-23 12:58:32.797548032     5695.246804
2020-06-23 15:25:52.183922176     8692.426644
2020-06-24 05:31:14.525965056     2575.392434
2020-06-24 06:43:43.109446912     3586.666598
2020-06-24 08:49:18.196271872     1540.093765
2020-06-24 14:50:24.490844928     3614.945816
2020-06-24 15:52:01.999751168     3658.978163
2020-06-25 02:09:41.400263168    26861.158005
2020-06-25 10:30:53.613639168    21391.419260
2020-06-25 11:06:41.691740160     1788.504132
2020-06-25 11:23:33.348498176     1011.656758
2020-06-25 11:42:07.755276800     1045.323695
2020-06-25 13:41:04.042957056     6846.911742
2020-06-25 14:22:29.130685952     2480.854122
2020-06-25 15:29:09.194838016     3893.022143
2020-06-25 16:11:45.240464896     1955.659670
2020-06-25 16:30:14.272532992     1089.232713
2020-06-25 18:23:38.591560960     6796.815644
2020-06-25 19:04:58.851831808     2475.189929
Name: diff, dtype: float64

## UPS 1

In [21]:
ups1=df1.copy()
ups1=ups1.set_index('time')
ups1 = ups1[ups1['DeviceID'] == 'EM4']
ups1['diff']=ups1.index.to_series().diff().dt.total_seconds().dropna()

In [22]:
ups1[ups1['diff'] > 60]['diff']

time
2020-06-23 06:57:29.963887104     120.986112
2020-06-23 06:59:58.630022912      64.275926
2020-06-23 07:05:57.048548096     168.499364
2020-06-23 08:15:57.647066112    2215.781276
2020-06-23 09:55:04.714277120     496.862434
2020-06-23 11:21:10.115379200      74.660500
2020-06-23 12:58:28.883273984    5691.064562
2020-06-23 15:25:53.529878016    8693.021755
2020-06-23 15:30:08.629041920     233.297966
2020-06-23 16:59:10.546034176     110.011483
2020-06-24 03:12:13.607183104     995.172807
2020-06-24 06:47:11.693496832    1000.206025
Name: diff, dtype: float64

## UPS 2

In [23]:
ups2=df1.copy()
ups2=ups2.set_index('time')
ups2 = ups2[ups2['DeviceID'] == 'EM5']
ups2['diff']=ups2.index.to_series().diff().dt.total_seconds().dropna()

In [24]:
ups2[ups2['diff'] > 60]['diff']

time
2020-06-23 06:57:31.238484992     122.508860
2020-06-23 07:00:00.224014080      65.602592
2020-06-23 07:05:59.955044096     172.414745
2020-06-23 07:34:57.933703936      68.270327
2020-06-23 08:16:04.231569920    2222.956026
2020-06-23 09:55:01.535660032     492.755168
2020-06-23 11:21:05.656920064      69.644946
2020-06-23 12:57:45.427680000    5646.975830
2020-06-23 15:25:51.116427776    8692.373270
2020-06-23 15:30:11.411241984     235.264789
2020-06-23 16:59:11.109900032     111.460161
2020-06-24 04:44:45.053835008     995.515300
2020-06-24 05:40:02.139130112    1000.595820
2020-06-24 06:01:20.364960000    1000.959030
2020-06-24 06:47:42.461014016    1000.458693
Name: diff, dtype: float64

In [45]:
df1['date']=df3['time'].dt.date

In [47]:
df1.groupby('date')['DeviceID'].unique()

date
2020-06-24          [EM9, EM5, EM6, EM7, EM1, EM23, EM10, EM4]
2020-06-25    [EM23, EM6, EM10, EM9, EM4, EM7, EM5, EM1, EM22]
Name: DeviceID, dtype: object